This is a markdown for interactive code

In [1]:
## Imports packages
import numpy as np
import pandas as pd
import os

In [2]:
# Change working directory
os.path.abspath('') # get current working directory
os.chdir('..') # change to parent directory
getdir = os.getcwd() # get current working directory after change

In [3]:
# Define input directory and initialize dictionaries
excel_files = os.listdir('Data/raw_data/ANH_gas_prod/') # create a list with all excel files names in the directory
datayear_dic = {} # create an empty dictionary to store excel per year dataframes
sheet_dict = {} # create an empty dictionary to store excel sheet dataframes
namelist = ['AÑO', 'MES', 'CAMPO', 'CONTRATO', 'EMPRESA', 'DEPARTAMENTO',
       'MUNICIPIO', 'PRODUCCION_FISCALIZADA', 'GASLIFT', 'GAS_REINYECTADO',
       'GAS_QUEMADO', 'CONSUMO_EN_CAMPO', 'ENVIADO_A_PLANTA', 'GAS_TRANSFORMADO',
       'ENTREGADO_A_GASEODUCTOS']

In [4]:
## Process each excel file and its sheets
for excel in excel_files: # loop through each excel file in the directory
    data_dir = f'Data/raw_data/ANH_gas_prod/{excel}' # create the path for each excel file
    sheet_names = pd.ExcelFile(data_dir).sheet_names[1:] # create a list with all sheet names except the first one

    for sheet in sheet_names: # loop through each sheet in the excel file
        tempdf = pd.read_excel(data_dir, sheet_name=sheet) # read each sheet into a dataframe and store it in the sheet_dict
        tempdf.columns =  namelist # assign the predefined column names to the dataframe
        sheet_dict[sheet] = tempdf

    datayear_dic[excel] = pd.concat(sheet_dict.values()) # concatenate all sheets dataframes into a single dataframe per excel file and store it in the datayear_dic

final_df = pd.concat(datayear_dic.values()) # concatenate all excel dataframes into a single final dataframe

In [5]:
datadir2 = 'Data/raw_data/ANH_gas_prod/Produccion_Fiscalizada_Gas_2013.xlsx'
raiz = pd.read_excel(datadir2, sheet_name= 'enero-13')

In [6]:
raiz

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,AGENCIA NACIONAL DE HIDROCARBUROS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"VICEPRESIDENCIA DE OPERACIONES, REGALIAS Y PAR...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BALANCE DE PRODUCCION DE GAS POR CAMPO (MILLON...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,enero de 2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,2013,1,CARUPANA,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PORE,0.41,0,0,0.41,0,0,0,0
270,2013,1,YAMÚ,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PAZ DE ARIPORO,0.63,0,0,0.63,0,0,0,0
271,TOTAL (MPC),NaN,NaN,NaN,NaN,NaN,NaN,94403.98,60.32,46063.24,2921.03,4105.59,9745.01,735.12,31508.92
272,PROMEDIO DIARIO (MPCPDC),NaN,NaN,NaN,NaN,NaN,NaN,3045.289677,1.945806,1485.910968,94.226774,132.438387,314.355161,23.713548,1016.416774


In [32]:
data2 = raiz.copy()
data2

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,AGENCIA NACIONAL DE HIDROCARBUROS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"VICEPRESIDENCIA DE OPERACIONES, REGALIAS Y PAR...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BALANCE DE PRODUCCION DE GAS POR CAMPO (MILLON...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,enero de 2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,2013,1,CARUPANA,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PORE,0.41,0,0,0.41,0,0,0,0
270,2013,1,YAMÚ,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PAZ DE ARIPORO,0.63,0,0,0.63,0,0,0,0
271,TOTAL (MPC),NaN,NaN,NaN,NaN,NaN,NaN,94403.98,60.32,46063.24,2921.03,4105.59,9745.01,735.12,31508.92
272,PROMEDIO DIARIO (MPCPDC),NaN,NaN,NaN,NaN,NaN,NaN,3045.289677,1.945806,1485.910968,94.226774,132.438387,314.355161,23.713548,1016.416774


In [87]:
data2 = raiz.copy()
while True:
    if data2.iloc[0,0] == 'AÑO':
        data2.columns = data2.iloc[0]
        data2 = data2.drop(0).reset_index(drop=True)
        break
    else:
        data2 = data2.drop(0).reset_index(drop=True)

dfshp = data2.shape
for i in range(0, dfshp[0]-1):
    if data2.iloc[i,0] != data2.iloc[0,0]:
        data2 = data2.drop([i])
        print(i)

265
266


In [85]:
range(0, dfshp[0]-1)

range(0, 267)

In [88]:
data2.tail()

,AÑO,MES,CAMPO,CONTRATO,EMPRESA,DEPARTAMENTO,MUNICIPIO,PRODUCCION \nFISCALIZADA,GAS LIFT,GAS\n REINYECTADO,GAS \nQUEMADO,CONSUMO \nEN CAMPO,ENVIADO \nA PLANTA,GAS \nTRANSFORMADO,ENTREGADO A \nGASEODUCTOS
261,2013,1,MONO ARAÑA,VMM 1,VETRA EXPLORACION Y PRODUCCION COLOMBIA S.A.,CESAR,AGUACHICA,0.8,0,0,0.8,0,0,0,0
262,2013,1,TRINIDAD,YALEA,PERENCO COLOMBIA LIMITED,CASANARE,TRINIDAD,2.95,0,0,2.95,0,0,0,0
263,2013,1,CARUPANA,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PORE,0.41,0,0,0.41,0,0,0,0
264,2013,1,YAMÚ,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PAZ DE ARIPORO,0.63,0,0,0.63,0,0,0,0
267,Fuente: ANH / Sistema Oficial de Liquidación y...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
data2

5,AÑO,MES,CAMPO,CONTRATO,EMPRESA,DEPARTAMENTO,MUNICIPIO,PRODUCCION \nFISCALIZADA,GAS LIFT,GAS\n REINYECTADO,GAS \nQUEMADO,CONSUMO \nEN CAMPO,ENVIADO \nA PLANTA,GAS \nTRANSFORMADO,ENTREGADO A \nGASEODUCTOS
5,AÑO,MES,CAMPO,CONTRATO,EMPRESA,DEPARTAMENTO,MUNICIPIO,PRODUCCION \nFISCALIZADA,GAS LIFT,GAS\n REINYECTADO,GAS \nQUEMADO,CONSUMO \nEN CAMPO,ENVIADO \nA PLANTA,GAS \nTRANSFORMADO,ENTREGADO A \nGASEODUCTOS
6,2013,1,ABANICO,ABANICO,KAPPA RESOURCES COLOMBIA LTD.,TOLIMA,ESPINAL,28.55,0,0,0.2,20.49,0,0,7.86
7,2013,1,AMBROSÍA,AMBROSÍA,INTEROIL COLOMBIA EXPLORATION AND PRODUCTION,TOLIMA,PIEDRAS,1.18,0,0,1.12,0.06,0,0,0
8,2013,1,APIAY,APIAY,ECOPETROL S.A,META,VILLAVICENCIO,112.61,0,0,9.33,0,103.28,22.99,0
9,2013,1,APIAY ESTE,APIAY,ECOPETROL S.A,META,VILLAVICENCIO,5.59,0,0,0.46,0,5.13,22.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,2013,1,CARUPANA,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PORE,0.41,0,0,0.41,0,0,0,0
270,2013,1,YAMÚ,YAMU,GEOPARK COLOMBIA PN S.A SUCURSAL COLOMBIA,CASANARE,PAZ DE ARIPORO,0.63,0,0,0.63,0,0,0,0
271,TOTAL (MPC),NaN,NaN,NaN,NaN,NaN,NaN,94403.98,60.32,46063.24,2921.03,4105.59,9745.01,735.12,31508.92
272,PROMEDIO DIARIO (MPCPDC),NaN,NaN,NaN,NaN,NaN,NaN,3045.289677,1.945806,1485.910968,94.226774,132.438387,314.355161,23.713548,1016.416774


In [ ]:
# Save the final dataframe to a csv file
final_df.to_csv('Data/processed_data/ANH_gas_concat.csv', index=False) # save the final dataframe to an csv file
#final_df.to_excel('Data/processed_data/ANH_gas_concat.xlsx', index=False) # save the final dataframe to an excel file
